# Kristins Alex 

## Imports

In [1]:
#pip install resnet

In [2]:
#pip install -U git+https://github.com/keras-team/keras git+https://github.com/keras-team/keras-applications

In [3]:
import os
from datetime import datetime
from dataclasses import dataclass
import warnings

import talos as ta
from talos.model import lr_normalizer
import numpy as np
from numpy.random import seed
import pandas as pd
import tensorflow as tf
import random

from tensorflow.compat.v1.keras import callbacks, backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.initializers import he_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess_input
from tensorflow.keras import Model

available_gpus = tf.config.experimental.list_physical_devices('GPU')
built_with_cuda = tf.test.is_built_with_cuda()

if not (not available_gpus) & built_with_cuda:
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    print("Num GPUs Available: ", len(available_gpus), "\n")
else:
    print("The installed version of TensorFlow {} does not include GPU support.\n".format(tf.__version__))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from numpy.random import seed

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.compat.v1.Session(config = config)
K.set_session(sess)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Using TensorFlow backend.


The installed version of TensorFlow 2.1.0 includes GPU support.

Num GPUs Available:  1 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4645685237463011593
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 20264236482
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6039782007414038193
physical_device_desc: "device: 0, name: GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6"
]
1 Physical GPUs, 1 Logical GPUs


In [4]:
def set_seed(s = 1):
    os.environ['PYTHONHASHSEED']='0'

    seed(s)
    tf.random.set_seed(s)
    random.seed(s)
    np.random.seed(s)
    
set_seed()
tf.compat.v1.disable_eager_execution()

## AlexNet

In [5]:
def alexnet(activation, leaky_alpha, dropout):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential([
        Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation=activation_layer, input_shape=(224,224,Global.num_image_channels)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Flatten(),
        Dense(4096, activation=activation_layer),
        Dropout(dropout),
        Dense(4096, activation=activation_layer),
        Dropout(dropout),
        Dense(units = 2, activation=activation_layer)
        #Dense(10, activation='softmax')
    ])
    return model

## VGG16

In [6]:
def vgg16_old(activation, leaky_alpha):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    
    model.add(Conv2D(input_shape=(224,224,Global.num_image_channels), filters = 64, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Flatten())
    model.add(Dense(units = 4096, activation = activation_layer))
    model.add(Dense(units = 4096, activation = activation_layer))
    #model.add(Dense(units = 2, activation = "softmax"))
    model.add(Dense(units = 2, activation=activation_layer))

    #opt = Adam(lr = 0.001)
    #model.compile(optimizer = opt, loss= keras.losses.categorical_crossentropy, metrics = ['accuracy'])
    #model.summary()
    return model

In [7]:
def vgg16_fc_like_RESNET(activation, leaky_alpha):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    cnn = VGG16(include_top=False, weights=None, input_tensor=None, input_shape=(224, 224, Global.num_image_channels))
    for layer in cnn.layers:
        layer.trainable = True
    x = cnn.output

    x =  AveragePooling2D((7, 7), padding='same')(x)
    x = Flatten()(x)

    x = Dense(units = 512, activation = activation_layer)(x)
    x = Dense(units = 256, activation = activation_layer)(x)
    x = Dense(units = 64, activation = activation_layer)(x)
    x = Dense(units = 2, activation = activation_layer)(x)
    model = Model(cnn.input, x)

    return model

In [8]:
def vgg16(activation, leaky_alpha, dropout_rate, first_neuron, hidden_layers):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    cnn = VGG16(include_top=False, weights=None, input_tensor=None, input_shape=(224, 224, Global.num_image_channels))
    for layer in cnn.layers:
        layer.trainable = True
    
    model.add(cnn)

    
    fc = Sequential()
    fc.add(Flatten(input_shape = model.output_shape[1:])) # (7, 7, 512)
    
    fc.add(Dense(units = first_neuron, kernel_initializer = glorot_uniform(seed = 1)))
    fc.add(activation_layer)
    if dropout_rate > 0.0:
        fc.add(Dropout(rate = dropout_rate))
    
    print('Number Hidden Layers {}'.format(hidden_layers))
    hidden_neuron_fraction = first_neuron
    for i in range(hidden_layers):
        hidden_neuron_fraction = hidden_neuron_fraction // 2
        fc.add(Dense(units = hidden_neuron_fraction, kernel_initializer = glorot_uniform(seed = 1)))
        fc.add(activation_layer)
        if dropout_rate > 0.0:
            fc.add(Dropout(rate = dropout_rate))
    
    fc.add(Dense(units = 2, kernel_initializer = glorot_uniform(seed = 1)))
    model.add(fc)
    
    for layer in model.layers:
        layer.trainable = True
    
    return model

In [9]:
def vgg16_he_uniform(activation, leaky_alpha):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    
    model.add(Conv2D(input_shape=(224,224,Global.num_image_channels), filters = 64, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Flatten())
    model.add(Dense(units = 4096, activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(Dense(units = 512, activation = activation_layer, kernel_initializer = he_uniform(seed = 1)))
    model.add(Dense(units = 2, activation=activation_layer, kernel_initializer = he_uniform(seed = 1)))

    return model

kernel_initializer = he_uniform(seed = 1)

##  MobileNet

In [10]:
def mobilenet(activation, leaky_alpha):
         
    activation_layer = ReLU()
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
        
    model = Sequential()
    cnn = MobileNetV2(include_top=False, weights=None, input_tensor=None, input_shape=(224, 224, Global.num_image_channels))
    for layer in cnn.layers:
        layer.trainable = True
    x = cnn.output

    x =  AveragePooling2D((7, 7), padding='same')(x)
    x = Flatten()(x)

    x = Dense(units = 512, activation = activation_layer)(x)
    x = Dense(units = 256, activation = activation_layer)(x)
    x = Dense(units = 64, activation = activation_layer)(x)
    x = Dense(units = 2, activation = activation_layer)(x)
    model = Model(cnn.input, x)

    return model

## ResNet

In [11]:
def resnet(activation, leaky_alpha):
         
    activation_layer = ReLU()
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
        
    model = Sequential()
    cnn = ResNet50(include_top=False, weights=None, input_tensor=None, input_shape=(224, 224, Global.num_image_channels))
    for layer in cnn.layers:
        layer.trainable = True
    x = cnn.output

    x =  AveragePooling2D((7, 7), padding='same')(x)
    x = Flatten()(x)

    x = Dense(units = 512, activation = activation_layer)(x)
    x = Dense(units = 256, activation = activation_layer)(x)
    x = Dense(units = 64, activation = activation_layer)(x)
    x = Dense(units = 2, activation = activation_layer)(x)
    model = Model(cnn.input, x)

    return model

## Generation of the DNN

In [12]:
def gen_net(x_train, y_train, x_val, y_val, params):
    
    K.clear_session()
    set_seed()
    
    train_generator, valid_generator = create_data_pipline(params['batch_size'], params['samples'])
    tg_steps_per_epoch = train_generator.n // train_generator.batch_size
    vg_validation_steps = valid_generator.n // valid_generator.batch_size
    print('Steps per Epoch: {}, Validation Steps: {}'.format(tg_steps_per_epoch, vg_validation_steps))
    
 
    if(Global.net_architecture == 'ALEX'):
        model = alexnet(params['activation'], params['leaky_alpha'], params['dropout'])
    elif(Global.net_architecture =='VGG16'):
        model = vgg16(params['activation'], params['leaky_alpha'], params['dropout'], params['first_neuron'], params['hidden_layers'])
        #model = vgg16(params['activation'], params['leaky_alpha'])
    elif(Global.net_architecture == 'RESNET'):
        model = resnet(params['leaky_alpha'], params['leaky_alpha'])
    elif(Global.net_architecture == 'MOBILENET'):
        model = mobilenet(params['leaky_alpha'], params['leaky_alpha'])
    else:
        print('Wrong net architecture!')
            
    model.compile(
        optimizer = params['optimizer'](lr = lr_normalizer(params['lr'], params['optimizer'])), 
        loss = Global.loss_function, 
        metrics = get_reduction_metric(Global.reduction_metric)
    )
    #print(model.summary())
    print('_________________________________________________________________')
    
    startTime = datetime.now()
    
    out = model.fit(
        x = train_generator,
        epochs = params['epochs'],
        validation_data = valid_generator,
        steps_per_epoch = tg_steps_per_epoch,
        validation_steps = vg_validation_steps,
        workers = 8
    )
    print("Time taken:", datetime.now() - startTime)

    return out, model

### Benutzerdefinierte Kostenfunktion & Metrik

In [13]:
def mean_absolut_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis = -1)

### Hilfsfunktion

In [14]:
def get_reduction_metric(metric):
    
    if metric == 'mean_absolut_error':
        return [mean_absolut_error]
    else:
        assert(False, 'Metric yet unknown - Please modify get_Reduction_Metric to meet your requirements')
        return None

### Struct for global parameter

In [15]:
@dataclass
class global_parameter:
    #========================================================
    # just change this, everything else will automaticlly adjusted
    
    net_architecture = 'MOBILENET' # 'ALEX' vs 'VGG16' vs 'RESNET'
    image_channels: str = 'rgb' # 'rgb' vs 'rgba'
    #======================================================== 
    loss_function: str = 'mean_squared_error'
    reduction_metric: str = 'mean_absolut_error'
    monitor_value: str = 'val_mean_absolut_error'
 
    dataset: str = '201129_2031'
    device: str = 'RTX_2080_Ti'
    data_augmentation: bool = False
    num_image_channels: int = 3
    image_dir: str = '..\\..\\data_generation\\dataset\\{}\\'.format(dataset)
    
    csv_file_name: str = 'labels_ks_RGB.csv'
    csv_file: str = image_dir + csv_file_name
    target_dir: str = '..\\output\\{}_{}_{}\\'.format(net_architecture, dataset, image_channels)
    results: str = '\\..\\{}_{}_Results.csv'.format(net_architecture, dataset)

        
Global = global_parameter

if(Global.image_channels == 'rgba'):
    Global.num_image_channels = 4
    Global.csv_file_name: str = 'labels_ks_RGBD.csv'
    Global.csv_file: str = Global.image_dir + Global.csv_file_name
    target_dir: str = '..\\output\\{}_{}_{}\\'.format(Global.net_architecture, Global.dataset, Global.image_channels)
    results: str = '\\..\\{}_{}_Results.csv'.format(Global.net_architecture, Global.dataset)
        
print()

### Generierung Datenpipeline (Angepasst für Talos)

In [16]:
def create_data_pipline(batch_size, num_samples):
    
    
    df = pd.read_csv(Global.csv_file)
    df_shuffled = df.sample(frac = 1, random_state = 1)
    df_train = df_shuffled[0 : int(num_samples * 0.8 // batch_size * batch_size)]
    df_valid = df_shuffled.drop(df_shuffled.index[0 : df_train.shape[0]])[0 : int(num_samples * 0.2 // batch_size * batch_size)]
    
    if Global.data_augmentation:
        train_data_generator = ImageDataGenerator(
            #rescale = 1./255,
            #preprocessing_function=vgg16_preprocess_input,
            preprocessing_function=mobilenet_preprocess_input,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.1,
            brightness_range = (0.5, 1.0), 
            fill_mode = 'nearest'
        )
    else:
        train_data_generator = ImageDataGenerator(
            #rescale = 1./255
            #preprocessing_function=vgg16_preprocess_input
            preprocessing_function=mobilenet_preprocess_input
        )
        
    train_generator = train_data_generator.flow_from_dataframe(
        dataframe = df_train,
        directory = Global.image_dir,
        x_col = 'Filename',
        y_col = ['Elevation', 'Azimuth'],
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = Global.image_channels,
        shuffle = True,
        seed = 1,
        batch_size = batch_size
    )
        
    valid_data_generator = ImageDataGenerator(
        #rescale = 1./255
        #preprocessing_function=vgg16_preprocess_input,
        preprocessing_function=mobilenet_preprocess_input,
    )
    
    valid_generator = valid_data_generator.flow_from_dataframe(
        dataframe = df_valid,
        directory = Global.image_dir,
        x_col = 'Filename',
        y_col = ['Elevation', 'Azimuth'],
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = Global.image_channels,
        shuffle = False,
        seed = 1,
        batch_size = batch_size
    )
    
    return train_generator, valid_generator

In [17]:

if(not os.path.exists(Global.target_dir)):
    os.makedirs(Global.target_dir)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(Global.target_dir))

device_file = open(Global.target_dir + '{}.txt'.format(Global.device), "a+")
device_file.close()

### GridSerach Parameter

In [18]:
#     Adam = RMSprop + Momentum (lr=0.001)
#     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
#     RMSprop = (lr=0.001)
#     SGD = (lr=0.01)
#     Adagrad
#
#hyper_parameter = {
#    'samples': [20000],
#    'epochs': [1],
#    'batch_size': [32, 64],
#    'optimizer': [Adam],
#    'lr': [1, 2, 5],
#    'first_neuron': [1024, 2048, 4096],
#    'dropout': [0.25, 0.50],
#    'activation': ['leakyrelu', 'relu'],
#    'hidden_layers': [0, 1, 2, 3, 4],
#    'leaky_alpha': [0.1] #Default bei LeakyReLU, sonst PReLU
#}

hyper_parameter = {
    'samples': [20000],
    'epochs': [1],
    'batch_size': [32],#, 64], 
    'optimizer': [Adam],
    'lr': [1, 2],#, 5],
    'first_neuron':  [1024, 2048, 4096],
    'dropout': [0.25, 0.50],
    'activation': ['relu', 'leakyrelu'],
    'hidden_layers': [0, 1, 2, 4],#, 3, 4],
    'leaky_alpha': [0.1] #Default bei LeakyReLU, sonst PReLU
}

### Start Talos Search

In [19]:
dummy_x = np.empty((1, 2, Global.num_image_channels, 224, 224))
dummy_y = np.empty((1, 2))

with tf.device('/device:GPU:0'):
    with warnings.catch_warnings():
            warnings.simplefilter("ignore")
    
            t = ta.Scan(
                x = dummy_x,
                y = dummy_y,
                model = gen_net,
                params = hyper_parameter,
                experiment_name = '{}'.format(Global.dataset),
                #shuffle=False,
                reduction_metric = Global.reduction_metric,
                disable_progress_bar = False,
                print_params = True,
                clear_session = True,
                save_weights = False
            )
        

t.data.to_csv(Global.target_dir + Global.results, index = True)
#t.data.to_csv(Global.target_dir + Global.results, index = True, mode='a', header=False)

  0%|                                                                                           | 0/96 [00:00<?, ?it/s]

{'activation': 'relu', 'batch_size': 32, 'dropout': 0.25, 'epochs': 1, 'first_neuron': 1024, 'hidden_layers': 0, 'leaky_alpha': 0.1, 'lr': 1, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 20000}
Found 16000 validated image filenames.
Found 4000 validated image filenames.
Steps per Epoch: 500, Validation Steps: 125
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
_________________________________________________________________
500/500 [==============================] - 1114s 2s/step - loss: 22680.7227 - mean_absolut_error: 112.6748 - val_loss: 22697.2094 - val_mean_absolut_error: 112.6631or: 11 - ETA: 31:34 - loss: 22459.9960 - - ETA: 18s - loss - ETA: 15s - loss: 22490.8343 - mean_absolut
Time taken: 0:18:35.718714


  1%|▊                                                                             | 1/96 [18:39<29:32:42, 1119.61s/it]

{'activation': 'relu', 'batch_size': 32, 'dropout': 0.25, 'epochs': 1, 'first_neuron': 1024, 'hidden_layers': 0, 'leaky_alpha': 0.1, 'lr': 2, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 20000}
Found 16000 validated image filenames.
Found 4000 validated image filenames.
Steps per Epoch: 500, Validation Steps: 125
_________________________________________________________________
500/500 [==============================] - 110s 219ms/step - loss: 22659.1242 - mean_absolut_error: 112.6114 - val_loss: 22697.2094 - val_mean_absolut_error: 112.6631
Time taken: 0:01:50.838587


  2%|█▋                                                                             | 2/96 [20:34<21:21:57, 818.28s/it]

...

 98%|█████████████████████████████████████████████████████████████████████████████▎ | 94/96 [3:25:50<04:03, 121.99s/it]

{'activation': 'leakyrelu', 'batch_size': 32, 'dropout': 0.5, 'epochs': 1, 'first_neuron': 4096, 'hidden_layers': 4, 'leaky_alpha': 0.1, 'lr': 1, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 20000}
Found 16000 validated image filenames.
Found 4000 validated image filenames.
Steps per Epoch: 500, Validation Steps: 125
_________________________________________________________________
500/500 [==============================] - 115s 230ms/step - loss: 21643.0621 - mean_absolut_error: 101.1580 - val_loss: 21575.0124 - val_mean_absolut_error: 100.2146
Time taken: 0:01:56.783114


 99%|██████████████████████████████████████████████████████████████████████████████▏| 95/96 [3:27:52<02:01, 121.83s/it]

{'activation': 'leakyrelu', 'batch_size': 32, 'dropout': 0.5, 'epochs': 1, 'first_neuron': 4096, 'hidden_layers': 4, 'leaky_alpha': 0.1, 'lr': 2, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 20000}
Found 16000 validated image filenames.
Found 4000 validated image filenames.
Steps per Epoch: 500, Validation Steps: 125
_________________________________________________________________
500/500 [==============================] - 116s 233ms/step - loss: 22680.7344 - mean_absolut_error: 112.6488 - val_loss: 22697.2094 - val_mean_absolut_error: 112.6631
Time taken: 0:01:57.917240


100%|███████████████████████████████████████████████████████████████████████████████| 96/96 [3:29:54<00:00, 131.19s/it]


In [20]:
t.data.to_csv(Global.target_dir + Global.results, index = True)
